* `docker pull zookeeper`
* `minikube start`
* `kubectl create -f namespace.yaml`
* `kubectl apply -f zk.yaml`
* `kubectl get deployment`
* `kubectl logs deployment/zookeeper-deployment`

In [10]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()
v1 = client.CoreV1Api()

In [9]:
namespace = 'zk'
print("Listing pods with their IPs:")
ret = v1.list_namespaced_pod(namespace)
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
10.244.0.210	zk	zookeeper-deployment-84c5bd-8lcnf
10.244.0.211	zk	zookeeper-deployment-84c5bd-vtths
10.244.0.212	zk	zookeeper-deployment-84c5bd-ztx4l


In [15]:
zk_pod = ret.items[0]

In [19]:
zk_pod.status.pod_ip

'10.244.0.210'

#### Connect to zookeeper

1. `minikube ip`
2. `kubectl get service zookeeper-loadbalancer -n zk` get lb external port number (31011) in my case

In [27]:
import subprocess

minikube_ip = subprocess.check_output(['minikube', 'ip'])
minikube_ip = str(minikube_ip, 'UTF-8').strip()

In [28]:
from kazoo.client import KazooClient
zk = KazooClient(hosts=f'{minikube_ip}:31011') # port number from kubectl get service command
zk.start()

In [30]:
zk.create('/test_node')
zk.create('/test_node/a', b"a simple string")

'/test_node/a'

In [32]:
zk.get('/test_node/a')

(b'a simple string',
 ZnodeStat(czxid=4, mzxid=4, ctime=1714377114589, mtime=1714377114589, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=15, numChildren=0, pzxid=4))

In [33]:
zk.stop()
zk.close()